-Libraries :  langchain langchain-openai streamlit python-dotenv

Streamlit is a Python library used to create interactive web applications for data science and machine learning. It is often used to build AI-powered meeting notes summarizers. These summarizers allow for the quick deployment of interfaces that can upload audio or text files, interact with large language models, and display structured summaries. 
Here is an overview of how to build or find Streamlit-based meeting summarizers:
Key Features of Streamlit Meeting Summarizers

File Uploading: Supports uploading audio (.mp3, .wav) or text (.txt, .pdf, .docx) files.

Transcription: Integrates with APIs like AssemblyAI or OpenAI Whisper to convert audio to text.

AI Summarization: Uses LangChain with models like GPT-4 or Gemini-1.5-flash to generate summaries, action items, and key takeaways.

Download Options: Allows users to export summaries as PDF, Word documents, or Markdown. 

Examples of Streamlit Apps
Meeting Notes Summarizer & Sharer (Groq): A tool that accepts text transcripts and generates bullet points for executives.

Meeting Minutes Extractor: Uses OpenAIs API to turn transcripts into structured markdown notes.

AI Meeting Summarizer (Assembly AI): Allows users to upload audio files, which are then transcribed and divided into chapters with summaries.

Live Meeting Summarizer: Provides speaker diarization and summarizes the conversation. 
Example Implementation (Backend Structure)

A typical Streamlit app for this purpose often involves:

UI Setup: import streamlit as st

File Loading: st.file_uploader()

Backend Integration: Using LangChain for prompt engineering and LLM interaction.

Display: st.markdown() or st.text_area() to show results. 
Common Tech Stack
Frontend/App Framework: Streamlit

LLM API: OpenAI (GPT-4o), Google Gemini, or Groq

Orchestration: LangChain

Speech-to-Text: Whisper or AssemblyAI 
These apps can be easily deployed on the Streamlit Community Cloud. 

In [15]:
#https://github.com/AlexisBalayre/AI-Powered-Meeting-Summarizer
from asyncio import subprocess
import requests
import streamlit as st
import gradio as gr
import json
import os

OLLAMA_SERVER_URL = "http://localhost:11434"  # Replace this with your actual Ollama server URL if different
WHISPER_MODEL_DIR = "/Users/vkdvamshi/gitRepos/whisper.cpp/models"  # Directory where whisper models are stored

def get_available_models() -> list[str]:
    """
    Retrieves a list of all available models from the Ollama server and extracts the model names.
    Returns:
        A list of model names available on the Ollama server.
    """
    response = requests.get(f"{OLLAMA_SERVER_URL}/api/tags")
    if response.status_code == 200:
        models = response.json()["models"]
        llm_model_names = [model["model"] for model in models]  # Extract model names
        return llm_model_names
    else:
        raise Exception(
            f"Failed to retrieve models from Ollama server: {response.text}"
        )

def get_available_whisper_models() -> list[str]:
    """
    Retrieves a list of available Whisper models based on downloaded .bin files in the whisper.cpp/models directory.
    Filters out test models and only includes official Whisper models (e.g., base, small, medium, large).

    Returns:
        A list of available Whisper model names (e.g., 'base', 'small', 'medium', 'large-V3').
    """
    # List of acceptable official Whisper models
    valid_models = ["base", "small", "medium", "large", "large-V3"]
    # Get the list of model files in the models directory
    model_files = [f for f in os.listdir(WHISPER_MODEL_DIR) if f.endswith(".bin")]
    # Filter out test models and models that aren't in the valid list
    whisper_models = [
        os.path.splitext(f)[0].replace("ggml-", "")
        for f in model_files
        if any(valid_model in f for valid_model in valid_models) and "test" not in f
    ]
    # Remove any potential duplicates
    whisper_models = list(set(whisper_models))
    return whisper_models

def summarize_with_model(llm_model_name: str, text: str) -> str:
    """
    Uses a specified model on the Ollama server to generate a summary.
    Handles streaming responses by processing each line of the response.
    Args:
        llm_model_name (str): The name of the model to use for summarization.
        context (str): Optional context for the summary, provided by the user.
        text (str): The transcript text to summarize.
    Returns:
        str: The generated summary text from the model.
    """
    prompt = f"""You are given a transcript from a meeting, along with some optional context.
    The transcript is as follows:
    {text}
    Please summarize the transcript."""
    headers = {"Content-Type": "application/json"}
    data = {"model": llm_model_name, "prompt": prompt}
    response = requests.post(f"{OLLAMA_SERVER_URL}/api/generate", json=data, headers=headers, stream=True
    )

    if response.status_code == 200:
        full_response = ""
        try:
            # Process the streaming response line by line
            for line in response.iter_lines():
                if line:
                    # Decode each line and parse it as a JSON object
                    decoded_line = line.decode("utf-8")
                    json_line = json.loads(decoded_line)
                    # Extract the "response" part from each JSON object
                    full_response += json_line.get("response", "")
                    # If "done" is True, break the loop
                    if json_line.get("done", False):
                        break
            return full_response
        except json.JSONDecodeError:
            print("Error: Response contains invalid JSON data.")
            return f"Failed to parse the response from the server. Raw response: {response.text}"
    else:
        raise Exception(
            f"Failed to summarize with model {llm_model_name}: {response.text}"
        )

# Translate and summarize function for text file input
def translate_and_summarize_textfile(
    text_file_path: str, llm_model_name: str
) -> tuple[str, str]:
    """
    Translates the text file into text using the whisper.cpp model and generates a summary using Ollama.
    Also provides the transcript file for download.
    Args:
        text_file_path (str): Path to the input text file.
        context (str): Optional context to include in the summary.
        llm_model_name (str): Model to use for summarizing the transcript.
    Returns:
        tuple[str, str]: A tuple containing the summary and the path to the transcript file for download.
    """
    # Read the output from the transcript
    with open(text_file_path, "r") as f:
        transcript = f.read()

    # Save the transcript to a downloadable file
    transcript_file = "transcript.txt"
    with open(transcript_file, "w") as transcript_f:
        transcript_f.write(transcript)
    # Generate summary from the transcript using Ollama's model
    summary = summarize_with_model(llm_model_name, transcript)
    # Clean up temporary files
    # Return the downloadable link for the transcript and the summary text
    return summary, transcript_file


# Gradio interface for Text file
def gradio_app_textfile(
    text_file, llm_model_name: str
) -> tuple[str, str]:
    """
    Gradio application to handle file upload, model selection, and summary generation.
    Args:
        text_file: The uploaded text file.
        context (str): Optional context provided by the user.
        whisper_model_name (str): The selected Whisper model name.
        llm_model_name (str): The selected language model for summarization.
    Returns:
        tuple[str, str]: A tuple containing the summary text and a downloadable transcript file.
    """
    print ( "llm_model_name:", llm_model_name)
    return translate_and_summarize_textfile(text_file, llm_model_name)

if __name__ == "__main__":
    # Retrieve available models for Gradio dropdown input
    ollama_models = get_available_models()  # Retrieve models from Ollama server
    whisper_models = (
        get_available_whisper_models()
    )  # Dynamically detect downloaded Whisper models
    iface = gr.Interface(
        fn=gradio_app_textfile,
        inputs=[
            # gr.Audio(type="filepath", label="Upload an audio file"),
            gr.File(
                label="Context (optional) Upload Text file",
                # placeholder="Provide any additional context for the summary",
                type="filepath",
            ),
            gr.Dropdown(
                choices=ollama_models,
                label="Select a model for summarization",
                value=ollama_models[0] if ollama_models else None,
            ),
        ],
        outputs=[
            gr.Textbox(
                label="Summary",
                # show_copy_button=True,
                # show_label=True,
            ),  # Display the summary generated by the Ollama model
            gr.File(
                label="Download Transcript"
            ),  # Provide the transcript as a downloadable file
        ],
        analytics_enabled=False,
        title="Meeting Summarizer",
        description="Upload an audio file of a meeting and get a summary of the key concepts discussed.",
    )
    
    iface.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.


llm_model_name: llama3.2:latest
Keyboard interruption in main thread... closing server.
